In [14]:
from keras.applications.inception_v3 import InceptionV3,preprocess_input
from keras.applications.vgg16 import VGG16,preprocess_input


from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

HEIGHT = 224
WIDTH = 224
TRAIN_DIR = "train_category"
BATCH_SIZE = 8

# create the base pre-trained model
#base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))

58892288/58889256 [==============================] - 88s 1us/step


In [15]:
from keras.preprocessing.image import ImageDataGenerator

In [27]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [17]:
train_generator = train_datagen.flow_from_directory(
        TRAIN_DIR,
        target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical')


Found 250 images belonging to 3 classes.


In [18]:
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model

def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

class_list = ["Kid's_wear", "Men's_wear", "Women's_wear"]
FC_LAYERS = [128, 128]
dropout = 0.5

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=len(class_list))


In [19]:
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint

NUM_EPOCHS = 15
BATCH_SIZE = 8


adam = Adam(lr=0.001)
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

filepath="checkpoints\\" + "VGG16" + "_model_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor=["acc"], verbose=1, mode='max')
callbacks_list = [checkpoint]

history = finetune_model.fit_generator(train_generator, epochs=NUM_EPOCHS, callbacks=callbacks_list,workers=6)


Epoch 1/15
32/32 [==============================] - 92s 3s/step - loss: 2.0419 - acc: 0.5004

Epoch 00001: saving model to checkpoints\VGG16_model_weights.h5
Epoch 2/15
32/32 [==============================] - 88s 3s/step - loss: 0.5160 - acc: 0.7615

Epoch 00002: saving model to checkpoints\VGG16_model_weights.h5
Epoch 3/15
32/32 [==============================] - 89s 3s/step - loss: 0.4980 - acc: 0.7498

Epoch 00003: saving model to checkpoints\VGG16_model_weights.h5
Epoch 4/15
32/32 [==============================] - 89s 3s/step - loss: 0.3308 - acc: 0.8632

Epoch 00004: saving model to checkpoints\VGG16_model_weights.h5
Epoch 5/15
32/32 [==============================] - 88s 3s/step - loss: 0.2800 - acc: 0.8671

Epoch 00005: saving model to checkpoints\VGG16_model_weights.h5
Epoch 6/15
32/32 [==============================] - 87s 3s/step - loss: 0.2806 - acc: 0.8831

Epoch 00006: saving model to checkpoints\VGG16_model_weights.h5
Epoch 7/15
32/32 [==============================] - 

In [34]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential, load_model

In [35]:
#Define Path
model_path = 'checkpoints/VGG16_model_weights.h5'
test_path = 'test_category'

#Load the pre-trained models
model = load_model(model_path)

In [46]:
import os
#Prediction Function
def predict(file):
    x = load_img(file, target_size=(224,224))
    x = img_to_array(x)
    x = np.expand_dims(x, axis=0)
    array = model.predict(x)
    result = array[0]
    #print(result)
    answer = np.argmax(result)
    if answer == 0:
        print("Predicted: Kid")
    elif answer == 1:
        print("Predicted: Men")
    elif answer == 2:
        print("Predicted: Women")
    return answer



In [55]:
import glob
files  = glob.glob(test_path+'/*')

In [56]:
#Walk the directory for every image
result_dict = {}
for ret in files:
    
    print(ret)
    result = predict(ret)
    result_dict[ret] = result

test_category\15.jpg
Predicted: Women
 
test_category\9.jpg
Predicted: Women
 
test_category\cf_5.jpg
Predicted: Men
 
test_category\s11.jpg
Predicted: Women
 
test_category\s15.jpg
Predicted: Women
 
test_category\SD_03_T25_3212M_B8_X_EC_90.jpg
Predicted: Men
 
test_category\SD_03_T25_3221M_P3_X_EC_90.jpg
Predicted: Men
 
test_category\SD_04_T77_3233N_A4_X_EC_90.jpg
Predicted: Kid
 
test_category\SD_04_T77_3262Q_J4_X_EC_90.jpg
Predicted: Kid
 
test_category\SD_04_T78_4209G_E4_X_EC_2.jpg
Predicted: Kid
 
test_category\SD_04_T87_1401T_KH_X_EC_90.jpg
Predicted: Kid
 


In [48]:
list(enumerate(os.listdir(test_path)))

[(0, '15.jpg'),
 (1, '9.jpg'),
 (2, 'cf_5.jpg'),
 (3, 's11.jpg'),
 (4, 's15.jpg'),
 (5, 'SD_03_T25_3212M_B8_X_EC_90.jpg'),
 (6, 'SD_03_T25_3221M_P3_X_EC_90.jpg'),
 (7, 'SD_04_T77_3233N_A4_X_EC_90.jpg'),
 (8, 'SD_04_T77_3262Q_J4_X_EC_90.jpg'),
 (9, 'SD_04_T78_4209G_E4_X_EC_2.jpg'),
 (10, 'SD_04_T87_1401T_KH_X_EC_90.jpg')]

In [33]:
test_generator.filenames

['f\\15.jpg',
 'f\\9.jpg',
 'f\\SD_03_T25_3212M_B8_X_EC_90.jpg',
 'f\\SD_03_T25_3221M_P3_X_EC_90.jpg',
 'f\\SD_04_T77_3233N_A4_X_EC_90.jpg',
 'f\\SD_04_T77_3262Q_J4_X_EC_90.jpg',
 'f\\SD_04_T78_4209G_E4_X_EC_2.jpg',
 'f\\SD_04_T87_1401T_KH_X_EC_90.jpg',
 'f\\cf_5.jpg',
 'f\\s11.jpg',
 'f\\s15.jpg']

In [32]:
predicted_class_indices

array([1, 2, 2, 2, 0, 0, 0, 1, 1, 0, 2], dtype=int64)

In [155]:
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model

def build_finetune_model(base_model, fc, dropout, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    
    # collar layer
    col_layer =  Dense(fc, activation='relu')(x)
    col_layer = Dropout(dropout)(col_layer)
    col_layer =  Dense(fc, activation='relu')(col_layer)
    col_predictions = Dense(num_classes['col'], activation='softmax', name='col_prediction')(col_layer)
    
    # sleeve layer
    sleeve_layer =  Dense(fc, activation='relu')(x)
    sleeve_layer = Dropout(dropout)(sleeve_layer)
    sleeve_layer =  Dense(fc, activation='relu')(sleeve_layer)
    sleeve_predictions = Dense(num_classes['sleeve'], activation='softmax', name='sleeve_prediction')(sleeve_layer) 
    
    # shirt_type
    shirt_layer =  Dense(fc, activation='relu')(x)
    shirt_layer = Dropout(dropout)(shirt_layer)
    shirt_layer =  Dense(fc, activation='relu')(shirt_layer)
    shirt_predictions = Dense(num_classes['shirt'], activation='softmax', name='shirt_prediction')(shirt_layer) 
    
    multihead_model = Model(inputs=base_model.input, outputs=[col_predictions,sleeve_predictions,shirt_predictions], name="fashionnet")

    return multihead_model

In [187]:
dropout = 0.5
num_classes = {'col':2,'sleeve':2,'shirt':2}
fc = 256


finetune_model = build_finetune_model(base_model, 
                                      fc = fc,
                                      dropout=dropout, 
                                      num_classes=num_classes)

print(finetune_model.summary())
losses = {'col_prediction':'categorical_crossentropy', 
          'sleeve_prediction':'categorical_crossentropy',
          'shirt_prediction':'categorical_crossentropy'}
lossWeights = {"col_prediction": 1.0, "sleeve_prediction": 1.0, "shirt_prediction": 1.0}
 

opt = Adam(lr=0.001)
finetune_model.compile(optimizer=opt, loss=losses, loss_weights=lossWeights, metrics=["accuracy"])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [188]:
y_trains = {
        "col_prediction": neck_encoded,
        "sleeve_prediction": sleeve_encoded,
        "shirt_prediction": shirt_type_encoded
}

In [189]:
filepath="checkpoints\\" + "VGG16" + "multioutput_model_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor=["loss"], verbose=1, mode='min')
callbacks_list = [checkpoint]

history = finetune_model.fit(train_X, y_trains, 
                        batch_size=8,
                        epochs=300,
                        callbacks=callbacks_list)

Epoch 1/300
84/84 [==============================] - 47s 560ms/step - loss: 7.7921 - col_prediction_loss: 2.3546 - sleeve_prediction_loss: 2.7863 - shirt_prediction_loss: 2.6512 - col_prediction_acc: 0.6071 - sleeve_prediction_acc: 0.6190 - shirt_prediction_acc: 0.5238

Epoch 00001: saving model to checkpoints\VGG16multioutput_model_weights.h5
Epoch 2/300
84/84 [==============================] - 36s 431ms/step - loss: 4.0792 - col_prediction_loss: 1.9704 - sleeve_prediction_loss: 0.6289 - shirt_prediction_loss: 1.4799 - col_prediction_acc: 0.6190 - sleeve_prediction_acc: 0.7976 - shirt_prediction_acc: 0.7024

Epoch 00002: saving model to checkpoints\VGG16multioutput_model_weights.h5
Epoch 3/300
84/84 [==============================] - 36s 429ms/step - loss: 1.8745 - col_prediction_loss: 0.8362 - sleeve_prediction_loss: 0.1684 - shirt_prediction_loss: 0.8700 - col_prediction_acc: 0.7500 - sleeve_prediction_acc: 0.9286 - shirt_prediction_acc: 0.7143

Epoch 00003: saving model to checkpoi

84/84 [==============================] - 34s 406ms/step - loss: 0.0533 - col_prediction_loss: 0.0212 - sleeve_prediction_loss: 0.0055 - shirt_prediction_loss: 0.0267 - col_prediction_acc: 1.0000 - sleeve_prediction_acc: 1.0000 - shirt_prediction_acc: 0.9881

Epoch 00047: saving model to checkpoints\VGG16multioutput_model_weights.h5
Epoch 48/300
84/84 [==============================] - 34s 408ms/step - loss: 0.0637 - col_prediction_loss: 0.0039 - sleeve_prediction_loss: 0.0232 - shirt_prediction_loss: 0.0366 - col_prediction_acc: 1.0000 - sleeve_prediction_acc: 0.9881 - shirt_prediction_acc: 0.9881

Epoch 00048: saving model to checkpoints\VGG16multioutput_model_weights.h5
Epoch 49/300
84/84 [==============================] - 34s 408ms/step - loss: 0.0412 - col_prediction_loss: 0.0143 - sleeve_prediction_loss: 0.0057 - shirt_prediction_loss: 0.0212 - col_prediction_acc: 0.9881 - sleeve_prediction_acc: 1.0000 - shirt_prediction_acc: 0.9881

Epoch 00049: saving model to checkpoints\VGG16m

KeyboardInterrupt: 

In [ ]:
plot_training(history)

# Plot the training and validation loss + accuracy
def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r.')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Training and validation accuracy')

    # plt.figure()
    # plt.plot(epochs, loss, 'r.')
    # plt.plot(epochs, val_loss, 'r-')
    # plt.title('Training and validation loss')
    plt.show()

    plt.savefig('acc_vs_epochs.png')

In [ ]:
'train_category\\mens_wear\\WW_collared_full\*'

In [89]:
import glob
def prepare_ww_data():
    
    dresses = glob.glob('WW\\WW_Dresses\\*')
    skirts = glob.glob('WW\\WW_Skirts\\*')
    
    women = glob.glob('WW\\*\\*')
    
    labelled_dresses = list(zip(dresses,len(dresses) * ['dresses']))
    labelled_skirts = list(zip(skirts,len(skirts) * ['skirts']))
    labelled_women = list(zip(women,len(women) * ['women']))
    
    
    labelled_women_categories = labelled_dresses+labelled_skirts
    
    return labelled_women, labelled_women_categories

In [90]:
glob.glob('train_category\\mens_wear\\WW_collared_full\*')

['train_category\\mens_wear\\WW_collared_full\\cf_1.jfif',
 'train_category\\mens_wear\\WW_collared_full\\cf_10.jfif',
 'train_category\\mens_wear\\WW_collared_full\\cf_11.jfif',
 'train_category\\mens_wear\\WW_collared_full\\cf_12.jfif',
 'train_category\\mens_wear\\WW_collared_full\\cf_2.jfif',
 'train_category\\mens_wear\\WW_collared_full\\cf_3.jfif',
 'train_category\\mens_wear\\WW_collared_full\\cf_4.jfif',
 'train_category\\mens_wear\\WW_collared_full\\cf_5.jfif',
 'train_category\\mens_wear\\WW_collared_full\\cf_6.jfif',
 'train_category\\mens_wear\\WW_collared_full\\cf_7.jfif',
 'train_category\\mens_wear\\WW_collared_full\\cf_8.jfif',
 'train_category\\mens_wear\\WW_collared_full\\cf_9.jfif']

In [67]:
attr = glob.glob("train_category\\Men's_wear\\Attributes\\*\\*\\*")

In [63]:
import pandas as pd

In [101]:
data = {}
for file in attr:
    file_name = file.split('\\')[5]
    if file_name not in data:
        data[file_name] = {'file_name':file_name, file.split('\\')[3]:file.split('\\')[4]}
    else:
        d = data[file_name]
        d[file.split('\\')[3]] = file.split('\\')[4]

In [103]:
df = pd.DataFrame(data)

In [106]:
df = df.transpose()

In [109]:
df.reset_index(inplace=True)

In [111]:
df.drop(columns='index',inplace=True)

In [112]:
df

,file_name,neck,shirt_type,sleeve
0,cf_1.jpg,collared_neck,t-shirt,full_sleeve
1,cf_10.jpg,collared_neck,t-shirt,full_sleeve
2,cf_11.jpg,collared_neck,t-shirt,full_sleeve
3,cf_12.jpg,collared_neck,t-shirt,full_sleeve
4,cf_2.jpg,collared_neck,t-shirt,full_sleeve
5,cf_3.jpg,collared_neck,t-shirt,full_sleeve
6,cf_4.jpg,collared_neck,t-shirt,full_sleeve
7,cf_5.jpg,collared_neck,t-shirt,full_sleeve
8,cf_6.jpg,collared_neck,t-shirt,full_sleeve
9,cf_7.jpg,collared_neck,t-shirt,full_sleeve


In [123]:
file = 'train_category/all/cf_1.jpg'
files = glob.glob('train_category/all/*')

In [169]:
for index, row in df.iterrows()

<generator object DataFrame.iterrows at 0x0000001D82D33CA8>

In [183]:
def load_images(files):
    X = []
    for index, row in df.iterrows():
        file = 'train_category/all/' + row['file_name']
        x = load_img(file, target_size=(224,224))
        x = img_to_array(x)
        x = x/255
        X.append(x)
    return X    
        

In [184]:
X = load_images(files)

In [185]:
train_X = np.array(X)

In [148]:
train_y_neck = np.array(df['neck'].tolist())
train_y_shirt_type = np.array(df['shirt_type'].tolist())
train_y_sleeve = np.array(df['sleeve'].tolist())

In [149]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

neck_label_encoder = LabelEncoder()
shirt_type_label_encoder = LabelEncoder()
sleeve_label_encoder = LabelEncoder()

neck_integer_encoded = neck_label_encoder.fit_transform(train_y_neck)
shirt_type_integer_encoded = neck_label_encoder.fit_transform(train_y_shirt_type)
sleeve_integer_encoded = neck_label_encoder.fit_transform(train_y_sleeve)

neck_encoded = to_categorical(neck_integer_encoded)
shirt_type_encoded = to_categorical(shirt_type_integer_encoded)
sleeve_encoded = to_categorical(sleeve_integer_encoded)
print(neck_encoded)
print(shirt_type_encoded)
print(sleeve_encoded)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]


In [176]:
train_X.shape

(84, 224, 224, 3)